# TP 4 - Procesamiento de Imagenes
Grupo 4 <br>
Integrantes:
- Pablo Smolkin
- Pedro Carranza Vélez
- Philippe Dutriez Diaz
- Francisco Daniel Ledesma

Nota: Las funciones utilizadas se encuentran en el archivo functions.py

In [1]:
from sys import path
from os.path import realpath
path.append(realpath("../")) # Makes it works everywhere
from common.functions import *

In [2]:
from functions import *

In [3]:
%load_ext autoreload
%autoreload 2

***
### 1. Dada la imagen *lena_gray.tif* se pide obtener:

* El negativo de la imagen.
* Una transformación tal que convierta los valores entre [0.25 0.5] al rango [0 1].

(Usar imadjust de Matlab pero implementarlo en Python)

***
### 2. Dada la imagen del globo ocular *eyebw.jpg*, buscar transformaciones que permitan resaltar las venas lo mejor posible.

***
### 3. Bit Plane Slicing
* Reproducir el ejemplo de la teoría del fractal 8 bit.
* Dada la imagen *onedollar.jpg*, convertirla a niveles de gris y mostrar todos los bit-planes (8 bit). A partir de los bit-planes obtenidos, reconstruir la imagen usando diferentes bit-planes (Experimentar usando los planos 8-7-6-5-1).

***
### 4. Explicar que es un histograma de una imagen. Escribir un código para calcularlo.

***
### 5. Explicar qué diferencia hay entre el brillo y contraste. Mostrar ejemplos.

***
### 6. Escribir un código que modifique el brillo y contraste de una imagen.

***
### 7.  ¿Para qué se usa la ecualización de un histograma? Dar ejemplos.

***
### 8. High Dynamic Range (HDR) Imaging
Investigar que es HDR y aplicarlo a las imágenes *office_x.jpg* (x=1-6).

Tiempos de exposicion : [0.0333, 0.1000, 0.3333, 0.6250, 1.3000, 4.0000]

Leer https://www.learnopencv.com/high-dynamic-range-hdr-imaging-using-opencv-cpp-python/